In [1]:
import numpy as np
import pandas as pd
import requests

In [2]:
df_names = ['confirmed_US', 'deaths_US', 'recovered_US']
df_list = [pd.DataFrame() for df in df_names]
df_dict = dict(zip(df_names, df_list))

In [3]:
url_part = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_'

In [4]:
for key, value in df_dict.items():
    value = pd.read_csv(url_part+key+'.csv', parse_dates=[0])
    
    value.rename(columns={'Province/State': 'Province_State', 'Country/Region': 'Country_Region'}, inplace = True)
    
    dim_col = value.columns[0:4]
    date_col = value.columns[4:]
    value = value.melt(id_vars = dim_col, value_vars = date_col, var_name = 'Date', value_name = key)
    
    value['Date'] = pd.to_datetime(value['Date'])

    df_dict[key] = value
    print(value.head())
    print(key+" cases dataframe created\n-----------------------------------------------")
    

ParserError: Unknown string format: FIPS

In [ ]:
join_on_col = ['Province_State','Country_Region','Lat','Long','Date'] 
df_COVID = df_dict['confirmed_global'].merge(df_dict['deaths_global'],
on=join_on_col, how='outer').merge(df_dict['recovered_global'], 
on=join_on_col, how='outer')
df_COVID.rename(columns = {'confirmed_global':'Confirmed', 'deaths_global':'Deaths', 'recovered_global':'Recovered'}, inplace = True)

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
df_COVID.sort_values(by = 'Country_Region', ascending = False).head(100)

In [ ]:
df_COVID.to_csv('COVID_TS.csv',index = False, header=True)

In [ ]:
import pygsheets

client = pygsheets.authorize(service_file='D:/DocsRepo/My Tableau Repository/Workbooks/Analyst Track/COVIDNYU-8d9c6d1a9db1.json')
print("----------------Authorized---------------")

In [ ]:
# Open the google spreadsheet (Where 'COVID-19' is the name of my sheet)
sheet = client.open('JHU')
print("----------------Sheet Opened---------------")

In [ ]:
#select the first sheet
wks = sheet[0]
print("-----------------------First Sheet Accessed----------------")

In [ ]:
# update the first sheet with df_COVID, starting at cell A1
wks.set_dataframe(df_COVID,(1,1))
print("-----------------------Data Updated--------------------")